<a href="https://colab.research.google.com/github/saurater/machine_learning/blob/main/Kaggle_Machine_Learning_Pipeline_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Machine Learning Pipeline I

**Adaptation:** Sam Faraday

# 1. Importando Bibliotecas

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error

# 2. Lendo o Dataset

In [2]:
data = pd.read_csv('melb_data.csv')

# 3. Separando o Alvo das Preditoras

In [3]:
y_alvo = data.Price
X_preditoras = data.drop(['Price'], axis=1)

# 4. Dividindo os Dados em Treino e Validação

In [4]:
X_preditoras_treino_completo, X_preditoras_validacao_completo, y_alvo_treino, y_alvo_validacao = \
train_test_split(X_preditoras, y_alvo, train_size=0.8, test_size=0.2, random_state=0)

# 5. Cardinalidade
Selecionando apenas as Colunas Categóricas com menos de 10 Valores Únicos

In [5]:
colunas_categoricas = [nome_coluna for nome_coluna in X_preditoras_treino_completo.columns \
                       if X_preditoras_treino_completo[nome_coluna].nunique() < 10 and  \
                       X_preditoras_treino_completo[nome_coluna].dtype == "object"]

# 6. Selecionando Colunas Numéricas

In [6]:
colunas_numericas = \
[nome_coluna for nome_coluna in X_preditoras_treino_completo.columns \
 if X_preditoras_treino_completo[nome_coluna].dtype in ['int64', 'float64']]

# 7. Unindo Categóricas e Numéricas

In [7]:
minhas_colunas = colunas_categoricas + colunas_numericas

X_preditoras_treino    = X_preditoras_treino_completo[minhas_colunas].copy()
X_preditoras_validacao = X_preditoras_validacao_completo[minhas_colunas].copy()

In [8]:
X_preditoras_treino.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


# 8. Preprocessando Dados Numéricos

In [9]:
# Preprocessing for numerical data
transformador_numerico = SimpleImputer(strategy='constant')

# 9. Preprocessando Dados Categóricos

In [10]:
transformador_categorico = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 10. Unindo os Preprocessamentos Categóricos e Numéricos

In [11]:
preprocessador = ColumnTransformer(
    transformers=[
        ('num', transformador_numerico, colunas_numericas),
        ('cat', transformador_categorico, colunas_categoricas)
    ])

# 11. Definindo o Modelo

In [12]:
modelo_floresta_randomica = RandomForestRegressor(n_estimators=100, random_state=0)

# Etapa 3: Criar e Avaliar o pipeline
Por fim, usamos a classe Pipeline para definir um pipeline que agrupa as etapas de pré-processamento e modelagem. Existem algumas coisas importantes a serem observadas:

Com o pipeline, pré-processamos os dados de treinamento e ajustamos o modelo em uma única linha de código. (Em contraste, sem um pipeline, temos que fazer imputação, codificação one-hot e treinamento de modelo em etapas separadas. Isso se torna especialmente confuso se tivermos que lidar com variáveis numéricas e categóricas!)

Com o pipeline, fornecemos os recursos não processados em X_preditoras_validacao para o comando predict(), e o pipeline pré-processa automaticamente os recursos antes de gerar as previsões. (No entanto, sem um pipeline, temos que nos lembrar de pré-processar os dados de validação antes de fazer previsões.)

# 12. Unindo Preprocessamento e Modelagem no Pipeline

In [13]:
meu_pipeline = Pipeline(steps=[('preprocessor', preprocessador),('model', modelo_floresta_randomica)])

# 13. Preprocessing of Training Data, Fit model 


In [14]:
meu_pipeline.fit(X_preditoras_treino, y_alvo_treino)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Rooms', 'Distance',
                                                   'Postcode', 'Bedroom2',
                                                   'Bathroom', 'Car',
                                                   'Landsize', 'BuildingArea',
                                                   'YearBuilt', 'Lattitude',
                                                   'Longtitude',
                                                   'Propertycount']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('oneh

# 14. Preprocessing of Validation Data, Get predictions


In [15]:
previsoes = meu_pipeline.predict(X_preditoras_validacao)

# 15. Evaluate the Model


In [16]:
score = round(mean_absolute_error(y_alvo_validacao, previsoes))
print('MAE:', score)

MAE: 160679
